# Recurrent Neural Networks (RNNs)

In Recurrent Neural Networks, data is processed the same way for every element in a sequence, and the output depends on the previous computations. This structure has proven to be very powerful for many applications, such as for natural language processing (NLP) and time series predictions.
![alt text][logo]

[logo]: https://github.com/sara-kassani/Python-Deep-Learning-Cookbook/blob/master/data/RNN.png?raw=true "RNNs"
As we can see in the figure, the output of a RNN does not onlydepend on the current input X t , but also on past inputs (X t-1 ). <br><br>
Basically, this gives the network a type of memory. ; There are multiple types of RNNs where the input and output
dimension can differ. An RNN can have one input variable and multiple output variables (1-to-n, with n > 1), multiple input variables and one output variable (n-to-1, with n > 1), or multiple input and output variables (n-to-m, with n,m > 1). 

# Implementing a simple RNN
We start with implementing a simple form of a recurrent neural network to understand the basic idea of RNNs. In this example, we will feed the RNN four binary variables. These represent the weather types on a certain day. For example, [1, 0, 0, 0] stands for sunny and [1, 0, 1, 0] stands for sunny and windy. The target value is a double representing the percentage of rain on that day. For this problem, we can say that the quantity of rain on a certain day also depends on the values of the previous day. This makes this problem well suited for a 4-to-1 RNN model.

In [1]:
import numpy as np

### Create the dummy dataset

In [2]:
X = []
X.append([1,0,0,0])
X.append([0,1,0,0])
X.append([0,0,1,0])
X.append([0,0,0,1])
X.append([0,0,0,1])
X.append([1,0,0,0])
X.append([0,1,0,0])
X.append([0,0,1,0])
X.append([0,0,0,1])

y = [0.20, 0.30, 0.40, 0.50, 0.05, 0.10, 0.20, 0.30, 0.40]

### For this regression problem, we will be using a sigmoid activation function

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_der(x):
    return 1.0 - x**2

### Next, we initialize the layers and weights:

In [4]:
layers = []
# 4 input variables, 10 hidden units and 1 output variable
n_units = (4, 16, 1)
n_layers = len(n_units)

layers.append(np.ones(n_units[0]+1+n_units[1]))
for i in range(1, n_layers):
    layers.append(np.ones(n_units[i]))

weights = []
for i in range(n_layers-1):
    weights.append(np.zeros((layers[i].size, layers[i+1].size)))

weights_delta = [0,]*len(weights)

### We are now ready to define the function for the forward pass:

In [5]:
def forwards(data):
    layers[0][:n_units[0]] = data
    layers[0][n_units[0]:-1] = layers[1]

    # Propagate the data forwards
    for i in range(1, n_layers):
        layers[i][...] = sigmoid(np.dot(layers[i-1], weights[i-1]))

    return layers[-1]

### In the backwards pass, we will determine the errors and update the weights:

In [6]:
def backwards(target, learning_rate=0.1, momentum=0.1):
    deltas = []
    error = target - layers[-1]
    delta = error * sigmoid_der(layers[-1])
    deltas.append(delta)

    # Determine error in hidden layers
    for i in range(n_layers-2, 0, -1):
        delta = np.dot(deltas[0], weights[i].T) * sigmoid_der(layers[i])
        deltas.insert(0, delta)

    # Update weights
    for i in range(len(weights)):
        layer = np.atleast_2d(layers[i])
        delta = np.atleast_2d(deltas[i])
        weights_delta_temp = np.dot(layer.T, delta)
        weights[i] += learning_rate*weights_delta_temp + momentum*weights_delta[i]
        weights_delta[i] = weights_delta_temp

    return (error**2).sum()

### Train our simple RNN:

In [7]:
n_epochs = 10000

for i in range(n_epochs):
    loss = 0
    for j in range(len(X)):
        forwards(X[j])
        backwards(y[j])
        loss += (y[j]-forwards(X[j]))**2
    if i%1000 == 0: print('epoch {} - loss: {:04.4f}'.format(i, loss[0]))

epoch 0 - loss: 0.3116
epoch 1000 - loss: 0.1660
epoch 2000 - loss: 0.1801
epoch 3000 - loss: 0.1877
epoch 4000 - loss: 0.1914
epoch 5000 - loss: 0.1922
epoch 6000 - loss: 0.1920
epoch 7000 - loss: 0.1916
epoch 8000 - loss: 0.1913
epoch 9000 - loss: 0.1912


### Print the results:

In [8]:
for i in range(len(X)):
    pred = forwards(X[i])
    loss = (y[i]-pred)**2
    print('X: {}; y: {:04.2f}; pred: {:04.2f}'.format(X[i], y[i], pred[0]))

X: [1, 0, 0, 0]; y: 0.20; pred: 0.15
X: [0, 1, 0, 0]; y: 0.30; pred: 0.39
X: [0, 0, 1, 0]; y: 0.40; pred: 0.29
X: [0, 0, 0, 1]; y: 0.50; pred: 0.34
X: [0, 0, 0, 1]; y: 0.05; pred: 0.30
X: [1, 0, 0, 0]; y: 0.10; pred: 0.16
X: [0, 1, 0, 0]; y: 0.20; pred: 0.38
X: [0, 0, 1, 0]; y: 0.30; pred: 0.30
X: [0, 0, 0, 1]; y: 0.40; pred: 0.33
